In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('vietnam_housing_dataset.csv')
df.head()

,Address,Area,Frontage,Access Road,House direction,Balcony direction,Floors,Bedrooms,Bathrooms,Legal status,Furniture state,Price
0,"Dự án The Empire - Vinhomes Ocean Park 2, Xã L...",84.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Have certificate,NaN,8.60
1,"Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...",60.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,7.50
2,"Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...",90.0,6.0,13.0,Đông - Bắc,Đông - Bắc,5.0,NaN,NaN,Sale contract,NaN,8.90
3,"Đường Nguyễn Văn Khối, Phường 11, Gò Vấp, Hồ C...",54.0,NaN,3.5,Tây - Nam,Tây - Nam,2.0,2.0,3.0,Have certificate,Full,5.35
4,"Đường Quang Trung, Phường 8, Gò Vấp, Hồ Chí Minh",92.0,NaN,NaN,Đông - Nam,Đông - Nam,2.0,4.0,4.0,Have certificate,Full,6.90


In [2]:
print(len(df["Address"].unique()))
print(len(df["Address"]))

10265
30229


In [3]:
df.describe()

,Area,Frontage,Access Road,Floors,Bedrooms,Bathrooms,Price
count,30229.000000,18665.000000,16932.000000,26626.000000,25067.000000,23155.000000,30229.000000
mean,68.498741,5.361692,7.853800,3.410426,3.511030,3.346837,5.872078
std,48.069835,4.346174,7.451313,1.328897,1.309116,1.400181,2.211877
min,3.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,40.000000,4.000000,4.000000,2.000000,3.000000,2.000000,4.200000
50%,56.000000,4.500000,6.000000,3.000000,3.000000,3.000000,5.900000
75%,80.000000,5.000000,10.000000,4.000000,4.000000,4.000000,7.500000
max,595.000000,77.000000,85.000000,10.000000,9.000000,9.000000,11.500000


# Xử lý ngoại lai


In [4]:
df = df[
    (df['Area'].between(20, 200))
    # (df['Frontage'].between(2, 30)) &
    # (df['Access Road'].between(2, 12)) &
    # (df['Floors'].between(1, 7)) &
    # (df['Bedrooms'].between(1, 8)) &
    # (df['Bathrooms'].between(1, 8))
]
print("\n✅ Số dòng sau khi loại bỏ ngoại lệ:", len(df))


✅ Số dòng sau khi loại bỏ ngoại lệ: 29406


# Xem dữ liệu bị thiếu

In [5]:
df.isnull().sum()   

Address                  0
Area                     0
Frontage             11236
Access Road          12927
House direction      20730
Balcony direction    24335
Floors                3340
Bedrooms              4955
Bathrooms             6834
Legal status          4351
Furniture state      13665
Price                    0
dtype: int64

# Xử lý dữ liệu bị thiếu

In [6]:
# Xóa 2 cột bị thiếu nhiều nhất vì dữ liệu thiếu > 70% 
df = df.drop(columns=["Balcony direction", "House direction"])

In [7]:
area_bins = [0,40,70,150,np.inf]
area_labels = ['Under 40m', '40-70m', '70-150m', 'Tren 150m']
df_area = pd.cut(df['Area'], bins=area_bins, labels=area_labels)

In [8]:
# Hàm điền median và điền mode theo gần nhất
fill_with_median = lambda x: x.fillna(x.median())
fill_with_mode = lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x

# Điền Floors theo Address + nhóm Area phụ
df['Floors'] = df.groupby(['Address', df_area])['Floors'].transform(fill_with_median)
df['Floors'] = df.groupby(['Address'])['Floors'].transform(fill_with_median)

# Điền Bedrooms theo Address + nhóm Area phụ
df['Bedrooms'] = df.groupby(['Address', df_area])['Bedrooms'].transform(fill_with_median)
df['Bedrooms'] = df.groupby(['Address'])['Bedrooms'].transform(fill_with_median)

# Điền Bathrooms theo Address + nhóm Area phụ
df['Bathrooms'] = df.groupby(['Address', df_area])['Bathrooms'].transform(fill_with_median)
df['Bathrooms'] = df.groupby(['Address'])['Bathrooms'].transform(fill_with_median)

# Điền Frontage theo Address + nhóm Area phụ
df['Frontage'] = df.groupby(['Address', df_area])['Frontage'].transform(fill_with_median)
df['Frontage'] = df.groupby(['Address'])['Frontage'].transform(fill_with_median)

df['Access Road'] = df.groupby('Address')['Access Road'].transform(fill_with_mode)

C:\Users\Dell\AppData\Local\Temp\ipykernel_3132\3507635475.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Floors'] = df.groupby(['Address', df_area])['Floors'].transform(fill_with_median)
C:\Users\Dell\AppData\Local\Temp\ipykernel_3132\3507635475.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Bedrooms'] = df.groupby(['Address', df_area])['Bedrooms'].transform(fill_with_median)
C:\Users\Dell\AppData\Local\Temp\ipykernel_3132\3507635475.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cur

In [9]:
df.isnull().sum()

Address                0
Area                   0
Frontage            2808
Access Road         3520
Floors              1099
Bedrooms            1356
Bathrooms           1750
Legal status        4351
Furniture state    13665
Price                  0
dtype: int64

In [10]:
# thay thế các giá trị NaN còn lại trong cột 'Floors' bằng 1
df['Floors'].fillna(1, inplace=True)

# thay thế các giá trị NaN còn lại trong cột 'Bedrooms' và 'Bathrooms' bằng 1
df['Bedrooms'].fillna(1,inplace=True)
df['Bathrooms'].fillna(1, inplace=True)

# Điền median cho các giá trị NaN còn lại trong cột 'Frontage' và 'Access Road'
df['Frontage'] = df['Frontage'].fillna(df['Frontage'].median())
df['Access Road'] = df['Access Road'].fillna(df['Access Road'].median())

C:\Users\Dell\AppData\Local\Temp\ipykernel_3132\862970219.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Floors'].fillna(1, inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_3132\862970219.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

In [11]:
# Điền giá trị thiếu trong cột 'Legal status' bằng 'Sale contract' và 'Furniture state' bằng 'N/A'
df['Legal status'].fillna('Sale contract', inplace=True) # sale contract không giấy sổ hồng chỉ hộp đồng mua bán thui
df['Furniture state'].fillna('No furnishings', inplace=True) # N/A là không có nội thất

C:\Users\Dell\AppData\Local\Temp\ipykernel_3132\100330148.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Legal status'].fillna('Sale contract', inplace=True) # sale contract không giấy sổ hồng chỉ hộp đồng mua bán thui
C:\Users\Dell\AppData\Local\Temp\ipykernel_3132\100330148.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

In [12]:
# Đổi kiểu dữ liệu của cột 'Bedrooms' và 'Bathrooms' từ float sang int bằng cách làm tròn xuống
df['Bedrooms'] = df['Bedrooms'].astype(float).apply(np.floor).astype(int)
df['Bathrooms'] = df['Bathrooms'].astype(float).apply(np.floor).astype(int)

# Xử lý dữ liệu bị trùng

In [13]:
print("\n🔁 Số dòng trùng lặp:", df.duplicated().sum())


🔁 Số dòng trùng lặp: 305


In [14]:
df = df.drop_duplicates()

print("✅ Số dòng sau khi xóa trùng:", len(df))

✅ Số dòng sau khi xóa trùng: 29101


# Xử lý ngoại lại

In [15]:
df = df[
    (df['Frontage'].between(2, 30)) &
    (df['Access Road'].between(2, 12)) &
    (df['Floors'].between(1, 7)) &
    (df['Bedrooms'].between(1, 8)) &
    (df['Bathrooms'].between(1, 8))
]
print("\n✅ Số dòng sau khi loại bỏ ngoại lệ:", len(df))


✅ Số dòng sau khi loại bỏ ngoại lệ: 25576


# Thêm 2 cột City và District từ cột Address

In [16]:
# Tách tỉnh/thành phố (phần cuối)
df["City"] = df["Address"].apply(lambda x: str(x).split(",")[-1].strip() if pd.notnull(x) else None)

# Tách quận/huyện (phần kế cuối)
df["District"] = df["Address"].apply(lambda x: str(x).split(",")[-2].strip() if pd.notnull(x) and len(str(x).split(",")) >= 2 else None)



# Chỉnh lại các Thành phố/Tỉnh cho thống nhất

In [17]:

import re

def clean_city(city):
    if pd.isna(city) or not str(city).strip():
        return np.nan

    city = str(city).strip().replace(".", "")
    
    # ❌ Loại bỏ các chuỗi chứa từ "giá" như "giá 6ty", "giá 2ty380", "giá 3 tỷ"
    if re.search(r"gi[aá]\s*\d", city.lower()):  
        return np.nan
       # Loại bỏ dòng không hợp lệ
    if not city or city.lower() == "phòng công chứng nguyễn thị thành":
        return np.nan
    invalid_cities = [
        "phòng công chứng nguyễn thị thành",
        "Quận Nam Từ Liêm",   # <- dòng này sẽ bị loại bỏ
        "TP Cam Ranh",
        "Đường số 11"
    ]
    if city in invalid_cities:
        return np.nan
    # if not city or city.lower() == "Quận Nam Từ Liêm":
    #     return np.nan
    # if not city or city.lower() == "TP Cam Ranh":
    #     return np.nan
    
    mapping = {
        "TP Hồ Chí Minh": "Hồ Chí Minh",
        "TP HCM": "Hồ Chí Minh",
        "TP. HCM": "Hồ Chí Minh",
        "TPHCM": "Hồ Chí Minh",
        "HCM": "Hồ Chí Minh",
        "HN": "Hà Nội",
        "Quận 8": "Hồ Chí Minh",
        "Quận Bình Thạnh": "Hồ Chí Minh",
        "Bán nhà chính chủ Phó Đức Chính khu Bà Chiểu - trung tâm Bình Thạnh giá cực tốt":"Hồ Chí Minh",
        "Hồ Chí Minh giá 2tỷ380":"Hồ Chí Minh",
        "TpHCM": "Hồ Chí Minh",
        "Hồ Chí Mính": "Hồ Chí Minh",
        "Bình Dương (gần cafe Xóm Vắng 2)": "Bình Dương",
    }

    if city in mapping:
        return mapping[city]
    return city

# 🔹 Làm sạch cột City
df["City"] = df["City"].apply(clean_city)

# 🔹 Xóa các dòng không hợp lệ (City bị NaN)
df = df.dropna(subset=["City"]).reset_index(drop=True)

# 🔹 Kiểm tra kết quả
print("Số địa phương duy nhất sau khi làm sạch:", len(df["City"].unique()))
print(sorted(df["City"].unique()))


Số địa phương duy nhất sau khi làm sạch: 55
['An Giang', 'Bà Rịa Vũng Tàu', 'Bình Dương', 'Bình Phước', 'Bình Thuận', 'Bình Định', 'Bắc Giang', 'Bắc Ninh', 'Bến Tre', 'Cà Mau', 'Cần Thơ', 'Gia Lai', 'Hà Nội', 'Hà Giang', 'Hà Nam', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình', 'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Hậu Giang', 'Hồ Chí Minh', 'Khánh Hòa', 'Kiên Giang', 'Long An', 'Lào Cai', 'Lâm Đồng', 'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận', 'Phú Thọ', 'Phú Yên', 'Quảng Nam', 'Quảng Ngãi', 'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Thừa Thiên Huế', 'Tiền Giang', 'Trà Vinh', 'Tây Ninh', 'Vĩnh Long', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên', 'Đà Nẵng', 'Đắk Lắk', 'Đồng Nai', 'Đồng Tháp']


# Chỉnh lại các Huyện/Quận cho thống nhất

In [18]:
import re

def clean_district(district):
    if not isinstance(district, str) or district.strip() == "":
        return None

    district = district.strip().replace(".", "")
    
    # Giữ lại chữ "Quận" nếu đi kèm số (VD: Quận 1, Quận 12)
    if re.match(r"(?i)^quận\s*\d+", district):
        return district.title()

    # Loại bỏ các prefix "Huyện", "Thành phố", "Thị xã"
    district = re.sub(r"(?i)^(huyện|thành phố|thị xã)\s+", "", district)

    # Nếu là "Quận" đi với chữ thì bỏ chữ "Quận"
    district = re.sub(r"(?i)^quận\s+", "", district)
    

    # Chuẩn hóa chữ cái đầu
    district = district.title()

    return district

# --- Ví dụ ---
# data = [
#     "Quận 1", "Quận Bình Thạnh", "Huyện Long Thành",
#     "Thành phố Dĩ An", "Thị xã Sa Pa", "Quận 12",
#     "Quận 9", "Quận Tân Bình", "Huyện Nhà Bè"
# ]


# df = pd.DataFrame({"District": data})
df["District"] = df["District"].apply(clean_district)

# Giữ lại các hàng mà District **không phải là phường/xã/thị trấn**
# Loại bỏ các hàng mà District là None trước
df = df[df['District'].notna()].copy()
df = df[~df['District'].str.lower().str.startswith(('phường', 'xã', 'thị trấn'))].copy()

print(df)


                                                 Address  Area  Frontage  \
0      Đường Nguyễn Văn Khối, Phường 11, Gò Vấp, Hồ C...  54.0      4.00   
1       Đường Quang Trung, Phường 8, Gò Vấp, Hồ Chí Minh  92.0      4.00   
2      Đường Phạm Văn Chiêu, Phường 14, Gò Vấp, Hồ Ch...  30.0      3.85   
3      Đường Quang Trung, Phường 12, Gò Vấp, Hồ Chí Minh  46.0      4.60   
4      Đường Ngô Tất Tố, Phường 22, Bình Thạnh, Hồ Ch...  60.0      3.50   
...                                                  ...   ...       ...   
25564  Đường Nguyễn Thái Sơn, Phường 5, Gò Vấp, Hồ Ch...  31.0      3.00   
25565   Đường Cô Giang, Phường 2, Phú Nhuận, Hồ Chí Minh  24.0      3.00   
25566  Đường Ngô Gia Tự, Phường Đức Giang, Long Biên,...  30.0      3.80   
25567  Đường Quang Trung, Phường 11, Gò Vấp, Hồ Chí Minh  96.0      4.60   
25568  Đường Đại lộ Thăng Long, Xã An Khánh, Hoài Đức...  35.0      4.65   

       Access Road  Floors  Bedrooms  Bathrooms      Legal status  \
0              3.5

In [19]:
df.to_csv('VN_House_price_Clean2.csv', index=False)